In [1]:
from data_processing import Dataset
from noise import NoiseScheduler
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
import torch
from torch.utils.data import TensorDataset, DataLoader
from diffusers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
from tqdm import tqdm
import torch
from torch.optim import Adam
from pathlib import Path
import os
import numpy as np

if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of available GPUs:", torch.cuda.device_count())
    print("Current GPU:", torch.cuda.current_device())
else:
    print("CUDA is not available. Running on CPU.")

/software/ys20884/miniconda3/envs/chip_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is available!
Number of available GPUs: 6
Current GPU: 0


In [2]:
dataset = Dataset(1_000, (120, 72), signal_file="data/CaloImages_signal.root", pile_up_file="data/CaloImages_bkg.root", save=False)

In [3]:
dataset() #once this is cached, you don't have to re-load

INFO:root:loading file data/CaloImages_signal.root
loading file data/CaloImages_signal.root
100%|██████████| 1000/1000 [00:01<00:00, 643.58it/s]
INFO:root:loading file data/CaloImages_bkg.root
loading file data/CaloImages_bkg.root
100%|██████████| 1000/1000 [00:01<00:00, 630.71it/s]


In [4]:
new_dim=(64,64)

In [5]:
dataset.preprocess(16, new_dim)

INFO:root:scaling
scaling
INFO:root:re-sizing
re-sizing


In [6]:
preprocess = transforms.Compose(
        [   
            transforms.ToTensor()
        ]
)

In [7]:
clean_frames = preprocess(dataset.signal).float().permute(1, 2, 0) #pytorch semantics
pile_up = preprocess(dataset.pile_up).float().permute(1, 2, 0)

In [8]:
batch_size = 16  # Adjust as needed",}
dataloader = DataLoader(clean_frames.unsqueeze(1), batch_size=batch_size, shuffle=False)

In [9]:
#check tensor shape

for batch in dataloader:
    for tensor in batch:
        print(tensor.shape)
        break
    break

torch.Size([1, 64, 64])


In [38]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        
        # Convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Upsample layer
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, x):
        # Forward pass through the layers
        x = self.conv1(x)
        x = self.pool(x)

        x = self.upsample(x)
        
        return x

# Create an instance of the model
model = SimpleModel()

# Print the model architecture
print(model)


SimpleModel(
  (conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (upsample): Upsample(scale_factor=2.0, mode=nearest)
)


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D

# Define the SimpleModel using Keras
model = Sequential()

# Convolutional layer
model.add(Conv2D(filters=1, kernel_size=(3, 3), padding='same', input_shape=(64, 64, 1)))

# Max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Upsample layer
model.add(UpSampling2D(size=(2, 2), interpolation='nearest'))

# Print the model architecture
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 64, 64, 1)         10        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 1)        0         
 2D)                                                             
                                                                 
 up_sampling2d_1 (UpSampling  (None, 64, 64, 1)        0         
 2D)                                                             
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.save('upsample_keras.h5')


In [39]:
from models import Model, TrainingConfig

#model = Model('UNet-lite', new_dim)
model = SimpleModel()

config = TrainingConfig(output_dir='trained_models_lite')

print(sum(p.numel() for p in model.parameters() if p.requires_grad)) #number of learnable params


10


In [40]:
dummy_input = torch.randn(1, 1, 64, 64)

# Pass the dummy input through the model
output = model(dummy_input)



In [41]:
from torch.fx import symbolic_trace

traced_model = symbolic_trace(model)

In [42]:
for node in traced_model.graph.nodes:
        print(node.args)

()
(x,)
(conv1,)
(pool,)
(upsample,)


In [43]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=len(dataloader) * config.num_epochs
)

In [44]:

def train_loop(config, model, noise_sample, optimizer, train_dataloader, lr_scheduler,noise_scheduler, n_events):
    

    global_step = torch.tensor(0)
    # Now you train the model
    for epoch in range(10):
        progress_bar = tqdm(total=len(train_dataloader))
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):

            clean_images = batch
            # Sample noise to add to the images
            
            bs = clean_images[0].shape[0]
            timesteps = torch.randint(
                0, config.num_train_timesteps, (bs,), device=clean_images.device
            ).long()

            random_seed = np.random.randint(0, n_events)

            noisy_images, noise_added = noise_scheduler.add_noise(clean_frame=clean_images, noise_sample=noise_sample, timestep=timesteps, random_seed=random_seed, n_events = n_events)

            # Predict the noise residual
            noise_pred = model(noisy_images)[0]
            loss = F.mse_loss(noise_pred, noise_added.float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            global_step += 1

            torch.save(model.state_dict(), os.path.join(config.output_dir, f"model_epoch_{epoch}.pt"))


In [47]:
from accelerate import notebook_launcher

args = (config, model, pile_up, optimizer, dataloader, lr_scheduler, NoiseScheduler('pile-up'), torch.tensor(1_000))

notebook_launcher(train_loop, args, num_processes=1) #will port to GPU if availible (can't train on mutli-GPU at Bristol) 

Launching training on one GPU.




/software/ys20884/miniconda3/envs/chip_env/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Using a target size (torch.Size([64, 64])) that is different to the input size (torch.Size([1, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
























/software/ys20884/miniconda3/envs/chip_env/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.













































Epoch 0: 100%|██████████| 63/63 [00:00<00:00, 129.09it/s, loss=0.0712, lr=4.88e-5, step=tensor(62)]







































































































































Epoch 1: 100%|██████████| 63/63 [00:00<00:00, 150.79it/s, loss